In [1]:
import requests
import numpy as np

In [2]:
# Tạo dữ liệu giả lập (150528 số float32)
data = np.random.rand(1, 3, 224, 224).astype(np.float32).flatten().tolist()

# API endpoint
url = "http://localhost:8000/v2/models/resnet50/infer"

# JSON payload
payload = {
    "inputs": [
        {
            "name": "data",
            "shape": [1, 3, 224, 224],
            "datatype": "FP32",
            "data": data
        }
    ]
}

# Gửi request
response = requests.post(url, json=payload)

# In kết quả
print(response.json())


{'model_name': 'resnet50', 'model_version': '1', 'outputs': [{'name': 'resnetv17_dense0_fwd', 'datatype': 'FP32', 'shape': [1, 1000], 'data': [-1.7193400859832764, 0.4843232035636902, 0.5464366674423218, 0.5255448222160339, 0.6679317951202393, 0.7165349721908569, 0.414615273475647, -0.22019942104816437, -1.3122996091842651, -0.14269909262657166, 3.373304843902588, 3.89656925201416, 2.899007558822632, 3.411736488342285, 2.938952922821045, 2.1400461196899414, 2.224985122680664, 1.7116739749908447, 2.6231110095977783, 2.6930906772613525, 0.4459402561187744, 2.6877613067626953, 2.4698193073272705, 1.9478759765625, 0.6320533752441406, -0.3333490490913391, 0.25099724531173706, 0.12607890367507935, -0.009652748703956604, -0.7218093872070312, -1.6812025308609009, 1.7537329196929932, -1.0240468978881836, 0.6540354490280151, 1.454535722732544, -1.938065767288208, -0.18589544296264648, -1.1687356233596802, 2.82077956199646, -0.5699526071548462, 1.6348881721496582, 1.1984738111495972, 1.6892437934

In [7]:
import numpy as np
import tritonclient.http as httpclient
import torchvision.transforms as transforms
from PIL import Image


In [10]:
client = httpclient.InferenceServerClient(url="localhost:8000")


In [17]:
# Tải ảnh từ file
image_path = "img.jpg"  # Thay bằng đường dẫn ảnh của bạn
image = Image.open(image_path).convert("RGB")  # Mở ảnh dưới dạng RGB

# Tiền xử lý ảnh (resize, normalize)
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # Resize ảnh về 224x224
    transforms.ToTensor(),           # Chuyển ảnh sang tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Chuẩn hóa ảnh theo ImageNet
])

# Áp dụng transform
transformed_img = transform(image).numpy()  # Chuyển tensor thành NumPy array
transformed_img = np.expand_dims(transformed_img, axis=0)  # Thêm batch dimension


In [20]:
# Chuẩn bị dữ liệu đầu vào
inputs = httpclient.InferInput("data", transformed_img.shape, datatype="FP32")
inputs.set_data_from_numpy(transformed_img, binary_data=True)

# Chuẩn bị output
outputs = httpclient.InferRequestedOutput("resnetv17_dense0_fwd", binary_data=True, class_count=1000)

In [21]:
# Gửi request đến Triton Server
results = client.infer(model_name="resnet50", inputs=[inputs], outputs=[outputs])

# Lấy kết quả
inference_output = results.as_numpy("resnetv17_dense0_fwd")
print(inference_output)

[[b'10.630373:90' b'7.904524:14' b'7.765345:92' b'7.014840:91'
  b'6.923308:11' b'6.848135:12' b'6.813577:136' b'6.214504:16'
  b'6.148381:88' b'5.400968:10' b'5.045775:95' b'4.995892:80'
  b'4.977252:94' b'4.898931:86' b'4.460465:13' b'4.394322:87'
  b'4.262871:96' b'4.142020:21' b'3.973354:325' b'3.875623:85'
  b'3.850911:82' b'3.769645:989' b'3.718749:83' b'3.675999:15'
  b'3.454314:42' b'3.258239:89' b'3.158980:17' b'3.103119:912'
  b'3.015244:952' b'2.983694:696' b'2.967816:7' b'2.772833:23'
  b'2.588412:522' b'2.512120:417' b'2.500723:559' b'2.493523:1'
  b'2.484808:946' b'2.437704:301' b'2.396608:323' b'2.371290:714'
  b'2.331350:317' b'2.234685:407' b'2.204053:392' b'2.177269:8'
  b'2.143024:448' b'2.113688:899' b'2.112277:19' b'2.104841:304'
  b'2.098799:64' b'2.096853:20' b'2.093580:326' b'2.090688:81'
  b'2.083930:641' b'2.077215:46' b'2.060114:59' b'2.056155:47'
  b'2.055085:337' b'2.044278:134' b'2.035674:966' b'2.030067:93'
  b'2.003760:357' b'1.966781:377' b'1.958288:321